In [2]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
db_user = ""
db_password = ""
db_host = ""  
db_port = "" 
db_name = ""
connection = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection)

In [4]:
query= (""" SELECT layerid, toid, SUM(datavalue) AS total_movements
FROM movements
GROUP BY layerid, toid
ORDER BY total_movements DESC;
""")

try:
    with engine.connect() as connection:
        chunks = pd.read_sql(query, connection, chunksize=1000) 
        df = pd.concat(chunks, ignore_index=True)  
    print(df)
except Exception as e:
    print(f"Errore durante l'esecuzione della query: {e}")

                   layerid                toid  total_movements
0       08|039|014|001|001  08|039|014|001|000           653495
1       08|099|013|000|002  08|099|013|000|001           641428
2       08|039|014|001|000  08|039|014|001|001           624069
3       08|099|013|000|001  08|099|013|000|002           605054
4       08|039|007|000|001  08|039|007|000|000           601716
...                    ...                 ...              ...
143990  08|099|027|000|000  08|040|032|000|000                3
143991  08|099|014|002|002  08|038|020|000|000                3
143992  08|099|014|002|002  08|038|015|000|000                3
143993  08|099|027|000|000  08|040|033|000|000                3
143994  08|099|014|002|002  08|038|010|000|000                3

[143995 rows x 3 columns]


In [5]:
# 1. Scarico solo le geometrie
query_geom = "SELECT layerid, geojson FROM geom_data"
df_geom = pd.read_sql(query_geom, engine)

# 2. Rinomino per i join successivi
df_geom_from = df_geom.rename(columns={'layerid': 'layerid', 'geojson': 'geo_from'})
df_geom_to = df_geom.rename(columns={'layerid': 'toid', 'geojson': 'geo_to'})

# 3. Faccio le due join in pandas
df_complete = df.merge(df_geom_from, on='layerid', how='left')
df_complete = df_complete.merge(df_geom_to, on='toid', how='left')
df_complete

,layerid,toid,total_movements,geo_from,geo_to
0,08|039|014|001|001,08|039|014|001|000,653495,"{'id': '5171f377-559d-4791-a57a-70a5b0e90c10',...","{'id': 'b192e047-3873-4003-8a52-9a2e568be68e',..."
1,08|099|013|000|002,08|099|013|000|001,641428,"{'id': '14b6a899-e430-4131-b11b-79d853ee8e56',...","{'id': '7667d236-7c84-4ea5-91df-8f5dae1c94d8',..."
2,08|039|014|001|000,08|039|014|001|001,624069,"{'id': 'b192e047-3873-4003-8a52-9a2e568be68e',...","{'id': '5171f377-559d-4791-a57a-70a5b0e90c10',..."
3,08|099|013|000|001,08|099|013|000|002,605054,"{'id': '7667d236-7c84-4ea5-91df-8f5dae1c94d8',...","{'id': '14b6a899-e430-4131-b11b-79d853ee8e56',..."
4,08|039|007|000|001,08|039|007|000|000,601716,"{'id': 'd1764910-2923-4aaa-a64e-ce8d9fb56949',...","{'id': '1a803bef-365b-4e43-b303-62c017828f3a',..."
...,...,...,...,...,...
143990,08|099|027|000|000,08|040|032|000|000,3,"{'id': 'c6f43962-b0b2-4d8e-a31f-c0b44d49209b',...","{'id': '74b8f3d7-f0d3-48e6-945f-be2f8a4d25fd',..."
143991,08|099|014|002|002,08|038|020|000|000,3,"{'id': 'b32bb6fc-c5f0-47b5-b338-5bb807d9a7f4',...","{'id': '8eb89b19-8905-49c4-8e3d-ead01f60e36e',..."
143992,08|099|014|002|002,08|038|015|000|000,3,"{'id': 'b32bb6fc-c5f0-47b5-b338-5bb807d9a7f4',...","{'id': '18ce5300-ad54-4ba5-a75d-53c56f993ccd',..."
143993,08|099|027|000|000,08|040|033|000|000,3,"{'id': 'c6f43962-b0b2-4d8e-a31f-c0b44d49209b',...","{'id': '82ee2c99-c53d-4662-83f1-7a8a45a7593c',..."


In [6]:
from shapely.geometry import shape
# 1. Crea dizionario layerid → (lat, lon)
geo_centroid_dict = {}

for _, row in df_geom.iterrows():
    try:
        geo = row['geojson']  # già dict!
        geom = shape(geo['geometry'])
        centroid = (geom.centroid.y, geom.centroid.x)
        geo_centroid_dict[row['layerid']] = centroid
    except Exception as e:
        print(f"Errore su {row['layerid']}: {e}")
        geo_centroid_dict[row['layerid']] = (None, None)

# 2. Crea nuove colonne facendo il mapping
df_complete['lat_from'] = df_complete['layerid'].map(lambda x: geo_centroid_dict.get(x, (None, None))[0])
df_complete['lon_from'] = df_complete['layerid'].map(lambda x: geo_centroid_dict.get(x, (None, None))[1])
df_complete['lat_to']   = df_complete['toid'].map(lambda x: geo_centroid_dict.get(x, (None, None))[0])
df_complete['lon_to']   = df_complete['toid'].map(lambda x: geo_centroid_dict.get(x, (None, None))[1])

LINEE 

In [8]:
from branca.colormap import LinearColormap
import matplotlib.colors as colors
import json
import folium
from jinja2 import Template
from branca.element import MacroElement

df_plot = df_complete.dropna(subset=['lat_from', 'lon_from', 'lat_to', 'lon_to'])
# Filtro per flussi significativi
df_plot = df_plot[df_plot['total_movements'] > 30000]

# Normalizza per peso e colore
min_val = df_plot['total_movements'].min()
max_val = df_plot['total_movements'].max()
norm = colors.Normalize(vmin=min_val, vmax=max_val)

# Colormap giallo → rosso scuro (YlOrRd)
colormap = LinearColormap(
    colors=['#ffffb2', '#fecc5c', '#fd8d3c', '#f03b20', '#bd0026'],
    vmin=min_val,
    vmax=max_val,
    caption="Numero di spostamenti"
)

# Crea la mappa centrata sull’Emilia-Romagna
mappa = folium.Map(
    location=[44.5, 11.3],
    zoom_start=8,
    tiles="CartoDB dark_matter"
)

# Legenda verticale personalizzata
legend_html = Template("""
{% macro html(this, kwargs) %}
<div style='
    position: fixed;
    top: 30px;         
    right: 30px;
    z-index:9999;
    background-color: rgba(0,0,0,0.8);
    padding: 10px;
    border-radius: 5px;
    font-size: 14px;
    color: white;
'>
    <b>Numero di spostamenti</b><br><br>
    <div style='line-height: 18px'>
        <div><span style='background:#f7fbff; width:15px; height:15px; display:inline-block; margin-right:5px;'></span>30.02k to 36.68k</div>
        <div><span style='background:#deebf7; width:15px; height:15px; display:inline-block; margin-right:5px;'></span>36.68k to 44.52k</div>
        <div><span style='background:#c6dbef; width:15px; height:15px; display:inline-block; margin-right:5px;'></span>44.52k to 55.8k</div>
        <div><span style='background:#9ecae1; width:15px; height:15px; display:inline-block; margin-right:5px;'></span>55.8k to 74.19k</div>
        <div><span style='background:#6baed6; width:15px; height:15px; display:inline-block; margin-right:5px;'></span>74.19k to 112.1k</div>
        <div><span style='background:#2171b5; width:15px; height:15px; display:inline-block; margin-right:5px;'></span>112.1k to 653.5k</div>
    </div>
</div>
{% endmacro %}
""")

class CustomVerticalLegend(MacroElement):
    def __init__(self):
        super().__init__()
        self._template = legend_html

mappa.get_root().add_child(CustomVerticalLegend())

# Funzione che assegna il colore esatto in base alla soglia
def get_color_by_value(value):
    if value <= 36680:
        return '#f7fbff'
    elif value <= 44520:
        return '#deebf7'
    elif value <= 55800:
        return '#c6dbef'
    elif value <= 74190:
        return '#9ecae1'
    elif value <= 112100:
        return '#6baed6'
    else:
        return '#2171b5'


# Aggiungi linee direzionali colorate e pesate
for _, row in df_plot.iterrows():
    value = row['total_movements']
    color = colormap(value)
    weight = max(1, int(8 * norm(value)))  # spessore dinamico da 1 a 8

    folium.PolyLine(
        locations=[
            (row['lat_from'], row['lon_from']),
            (row['lat_to'], row['lon_to'])
        ],
        weight=weight,
        color = get_color_by_value(value),
        opacity=0.8,
        tooltip=f"{row['layerid']} → {row['toid']}: {value} spostamenti"
    ).add_to(mappa)

# Salva la mappa
mappa.save("mappageneraleLINEE.html")


ARCHI

In [10]:
from branca.colormap import StepColormap
from branca.element import MacroElement
from geopy.distance import geodesic

def arc_coordinates(p1, p2, num_points=200, base_height=0.002):
    distance_km = geodesic(p1, p2).km
    height = min(0.05, base_height * distance_km)

    lats = np.linspace(p1[0], p2[0], num_points)
    lons = np.linspace(p1[1], p2[1], num_points)
    curve = height * np.sin(np.linspace(0, np.pi, num_points))
    lats += curve
    return list(zip(lats, lons))


# Filtro flussi
df_plot = df_plot[df_plot['total_movements'] > 30000]


# Mappa con sfondo scuro
mappa = folium.Map(
    location=[44.5, 11.3],
    zoom_start=8,
    tiles="CartoDB dark_matter"
)

# Legenda verticale personalizzata
legend_html = Template("""
{% macro html(this, kwargs) %}
<div style='
    position: fixed;
    top: 30px;         
    right: 30px;
    z-index:9999;
    background-color: rgba(0,0,0,0.8);
    padding: 10px;
    border-radius: 5px;
    font-size: 14px;
    color: white;
'>
    <b>Numero di spostamenti</b><br><br>
    <div style='line-height: 18px'>
        <div><span style='background:#f7fbff; width:15px; height:15px; display:inline-block; margin-right:5px;'></span>30.02k to 36.68k</div>
        <div><span style='background:#deebf7; width:15px; height:15px; display:inline-block; margin-right:5px;'></span>36.68k to 44.52k</div>
        <div><span style='background:#c6dbef; width:15px; height:15px; display:inline-block; margin-right:5px;'></span>44.52k to 55.8k</div>
        <div><span style='background:#9ecae1; width:15px; height:15px; display:inline-block; margin-right:5px;'></span>55.8k to 74.19k</div>
        <div><span style='background:#6baed6; width:15px; height:15px; display:inline-block; margin-right:5px;'></span>74.19k to 112.1k</div>
        <div><span style='background:#2171b5; width:15px; height:15px; display:inline-block; margin-right:5px;'></span>112.1k to 653.5k</div>
    </div>
</div>
{% endmacro %}
""")


class CustomVerticalLegend(MacroElement):
    def __init__(self):
        super().__init__()
        self._template = legend_html

mappa.get_root().add_child(CustomVerticalLegend())

# Normalizzazione per spessore
norm = colors.Normalize(vmin=min_val, vmax=max_val)

# Funzione che assegna il colore esatto in base alla soglia
def get_color_by_value(value):
    if value <= 36680:
        return '#f7fbff'
    elif value <= 44520:
        return '#deebf7'
    elif value <= 55800:
        return '#c6dbef'
    elif value <= 74190:
        return '#9ecae1'
    elif value <= 112100:
        return '#6baed6'
    else:
        return '#2171b5'

# Aggiunta archi
for _, row in df_plot.iterrows():
    value = row['total_movements']
    color = get_color_by_value(value)
    weight = 1 + 4 * norm(value)

    p1 = (row['lat_from'], row['lon_from'])
    p2 = (row['lat_to'], row['lon_to'])
    coords = arc_coordinates(p1, p2)

    folium.PolyLine(
        locations=coords,
        weight=weight,
        color = get_color_by_value(value),
        opacity=0.5,
        tooltip=f"{row['layerid']} → {row['toid']}: {value} spostamenti"
    ).add_to(mappa)

# Salva
mappa.save("mappageneraleARCHI.html")